In [5]:
#Umbrella Problem
from collections import deque
import copy

class State:
    def __init__(self, people, total_time, umbrella_side, path=None):
        self.people = people  
        self.total_time = total_time
        self.umbrella_side = umbrella_side
        self.path = path if path is not None else []
    
    def __eq__(self, other):
        if not isinstance(other, State):
            return False
        return (self.people == other.people and 
                self.umbrella_side == other.umbrella_side)
    
    def __hash__(self):
        people_tuple = tuple((name, (info[0], info[1])) for name, info in sorted(self.people.items()))
        return hash((people_tuple, self.umbrella_side))
    
    def is_goal(self):
        return all(info[1] == 'R' for info in self.people.values()) and self.total_time <= 60
    
    def get_possible_moves(self):
        moves = []
        current_side = self.umbrella_side
        
        current_side_people = [name for name, info in self.people.items() if info[1] == current_side]
        
        for i in range(len(current_side_people)):
            for j in range(i+1, len(current_side_people)):
                person1 = current_side_people[i]
                person2 = current_side_people[j]
                move_time = max(self.people[person1][0], self.people[person2][0])
                
                new_people = copy.deepcopy(self.people)
                new_people[person1] = (new_people[person1][0], 'R' if current_side == 'L' else 'L')
                new_people[person2] = (new_people[person2][0], 'R' if current_side == 'L' else 'L')
                
                new_state = State(
                    new_people,
                    self.total_time + move_time,
                    'R' if current_side == 'L' else 'L',
                    self.path + [f"{person1} and {person2} cross to {'R' if current_side == 'L' else 'L'} (time: {move_time})"]
                )
                
                opposite_side = 'R' if current_side == 'L' else 'L'
                opposite_side_people = [name for name, info in new_people.items() if info[1] == opposite_side]
                
                for person in opposite_side_people:
                    return_time = new_people[person][0]
                    
                    return_people = copy.deepcopy(new_people)
                    return_people[person] = (return_people[person][0], current_side)
                    
                    return_state = State(
                        return_people,
                        new_state.total_time + return_time,
                        current_side,
                        new_state.path + [f"{person} returns to {current_side} (time: {return_time})"]
                    )
                    
                    moves.append(return_state)
                
                if new_state.is_goal():
                    moves.append(new_state)
        
        return moves
    
    def __str__(self):
        return f"Time: {self.total_time}, Umbrella: {self.umbrella_side}, People: {self.people}"
        
#BFS Code
def bfs(initial_state):
    visited = set()
    queue = deque()
    queue.append(initial_state)
    visited.add(initial_state)
    
    while queue:
        current_state = queue.popleft()
        
        if current_state.is_goal():
            return current_state
        
        for next_state in current_state.get_possible_moves():
            if next_state not in visited and next_state.total_time <= 60:
                visited.add(next_state)
                queue.append(next_state)
    
    return None

#DFS Code
def dfs(initial_state, visited=None):
    if visited is None:
        visited = set()
        
    if initial_state.is_goal():
        return initial_state

    visited.add(initial_state)

    for next_state in initial_state.get_possible_moves():
        if next_state not in visited and next_state.total_time <= 60:
            result = dfs(next_state, visited)
            if result:
                return result
    
    return None


people = {
    "amogh": (5, 'L'),
    "ameya": (10, 'L'),
    "gma": (20, 'L'),
    "gfa": (25, 'L')
}

initial_state = State(people, 0, 'L')
solution = bfs(initial_state)

if solution:
    print("->Solution found using BFS!")
    print(f"Total time: {solution.total_time} minutes")
    print("\nSteps:")
    for i, step in enumerate(solution.path, 1):
        print(f"{i}. {step}")
else:
    print("No solution found within 60 minutes.")

solution = dfs(initial_state)

if solution:
    print("\n->Solution found using DFS!")
    print(f"Total time: {solution.total_time} minutes")
    print("\nSteps:")
    for i, step in enumerate(solution.path, 1):
        print(f"{i}. {step}")
else:
    print("No solution found within 60 minutes.")

->Solution found using BFS!
Total time: 60 minutes

Steps:
1. amogh and ameya cross to R (time: 10)
2. amogh returns to L (time: 5)
3. gma and gfa cross to R (time: 25)
4. ameya returns to L (time: 10)
5. amogh and ameya cross to R (time: 10)

->Solution found using DFS!
Total time: 60 minutes

Steps:
1. amogh and ameya cross to R (time: 10)
2. amogh returns to L (time: 5)
3. gma and gfa cross to R (time: 25)
4. ameya returns to L (time: 10)
5. amogh and ameya cross to R (time: 10)


In [10]:
#RABBIT PROBLEM

from collections import deque
class RabbitState:
    def __init__(self, positions):
        self.positions = positions.copy()
        self.empty_pos = positions.index('_')
    
    def goal_test(self):
        return self.positions == ['W', 'W', 'W', '_', 'E', 'E', 'E']
    
    def is_valid(self):
        return self.positions.count('E') == 3 and self.positions.count('W') == 3 and '_' in self.positions
    
    def move_gen(self):
        children = []
        empty_pos = self.empty_pos
        
        if empty_pos > 0 and self.positions[empty_pos - 1] == 'E':
            new_pos = self.positions.copy()
            new_pos[empty_pos], new_pos[empty_pos - 1] = new_pos[empty_pos - 1], new_pos[empty_pos]
            children.append(RabbitState(new_pos))
            
        if empty_pos > 1 and self.positions[empty_pos - 2] == 'E' and self.positions[empty_pos - 1] == 'W':
            new_pos = self.positions.copy()
            new_pos[empty_pos], new_pos[empty_pos - 2] = new_pos[empty_pos - 2], new_pos[empty_pos]
            children.append(RabbitState(new_pos))
        
        if empty_pos < len(self.positions) - 1 and self.positions[empty_pos + 1] == 'W':
            new_pos = self.positions.copy()
            new_pos[empty_pos], new_pos[empty_pos + 1] = new_pos[empty_pos + 1], new_pos[empty_pos]
            children.append(RabbitState(new_pos))
        
        if empty_pos < len(self.positions) - 2 and self.positions[empty_pos + 2] == 'W' and self.positions[empty_pos + 1] == 'E':
            new_pos = self.positions.copy()
            new_pos[empty_pos], new_pos[empty_pos + 2] = new_pos[empty_pos + 2], new_pos[empty_pos]
            children.append(RabbitState(new_pos))
        
        return children
    
    def __str__(self):
        return ' '.join(self.positions)
    
    def __eq__(self, other):
        return self.positions == other.positions
    
    def __hash__(self):
        return hash(tuple(self.positions))

def BFS():
    initial_state = RabbitState(['E', 'E', 'E', '_', 'W', 'W', 'W'])
    
    if initial_state.goal_test():
        return [initial_state]
    
    queue = deque()
    queue.append((initial_state, []))
    visited = set()
    visited.add(initial_state)
    
    while queue:
        current_state, path = queue.popleft()
        
        for next_state in current_state.move_gen():
            if next_state not in visited:
                if next_state.goal_test():
                    return path + [current_state, next_state]
                visited.add(next_state)
                new_path = path + [current_state]
                queue.append((next_state, new_path))
    
    return None  

def DFS():
    initial_state = RabbitState(['E', 'E', 'E', '_', 'W', 'W', 'W'])
    
    if initial_state.goal_test():
        return [initial_state]
    
    stack = [(initial_state, [])]
    visited = set()
    visited.add(initial_state)
    
    while stack:
        current_state, path = stack.pop()
        
        for next_state in current_state.move_gen():
            if next_state not in visited:
                if next_state.goal_test():
                    return path + [current_state, next_state]
                visited.add(next_state)
                new_path = path + [current_state]
                stack.append((next_state, new_path))
    
    return None

def print_solution(solution):
    """Print the solution path"""
    if not solution:
        print("No solution found")
        return
    
    print("Solution found in", len(solution)-1, "moves:")
    for i in range(len(solution)-1):
        current = solution[i]
        next_state = solution[i+1]
        
        current_empty = current.empty_pos
        next_empty = next_state.empty_pos
        moved_rabbit = next_state.positions[current_empty]
        
        print(f"{i+1}. Move {moved_rabbit} from position {current_empty + 1} to {next_empty + 1}")
        print("   State:", next_state)

print("BFS Solution:")
bfs_solution = BFS()
print_solution(bfs_solution)

print("\nDFS Solution:")
dfs_solution = DFS()
print_solution(dfs_solution)

BFS Solution:
Solution found in 15 moves:
1. Move E from position 4 to 3
   State: E E _ E W W W
2. Move W from position 3 to 5
   State: E E W E _ W W
3. Move W from position 5 to 6
   State: E E W E W _ W
4. Move E from position 6 to 4
   State: E E W _ W E W
5. Move E from position 4 to 2
   State: E _ W E W E W
6. Move E from position 2 to 1
   State: _ E W E W E W
7. Move W from position 1 to 3
   State: W E _ E W E W
8. Move W from position 3 to 5
   State: W E W E _ E W
9. Move W from position 5 to 7
   State: W E W E W E _
10. Move E from position 7 to 6
   State: W E W E W _ E
11. Move E from position 6 to 4
   State: W E W _ W E E
12. Move E from position 4 to 2
   State: W _ W E W E E
13. Move W from position 2 to 3
   State: W W _ E W E E
14. Move W from position 3 to 5
   State: W W W E _ E E
15. Move E from position 5 to 4
   State: W W W _ E E E

DFS Solution:
Solution found in 15 moves:
1. Move W from position 4 to 5
   State: E E E W _ W W
2. Move E from position 5 to 